# Molecular dynamics tutorial: protein in water

## Step 1: creando la topologia 

In [ ]:
%%bash
gmx_mpi='/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi'
echo -e "15" | $gmx_mpi pdb2gmx -f protein.pdb -o protein.gro -water spce

## Step 2: solvatar 

In [ ]:
%%bash
gmx_mpi='/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi'
$gmx_mpi editconf -f protein.gro -o protein_newbox.gro -c -d 1.0 -bt cubic

In [ ]:
!vmd gp50_newbox.gro

In [ ]:
%%bash
gmx_mpi='/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi'
$gmx_mpi solvate -cp protein_newbox.gro -cs spc216.gro -o protein_solv.gro -p topol.top

In [ ]:
!vmd gp50_solv.gro

In [ ]:
%%bash 
pip install gdown 
gdown https://drive.google.com/uc?id=1JVgz07QGz33vhmsWB5nAGGrHIGuwfeYE
unzip mdp_files_only_water.zip && rm mdp_*

## Step 3: agregando iones

In [ ]:
%%bash
gmx_mpi='/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi'
$gmx_mpi grompp -f ions.mdp -c protein_solv.gro -p topol.top -o ions.tpr

In [ ]:
%%bash
gmx_mpi='/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi'
echo -e "13"$ | $gmx_mpi genion -s ions.tpr -o protein_solv_ions.gro -p topol.top -pname NA -nname CL -neutral

## Step 4: minimizacion

In [ ]:
%%bash
gmx_mpi='/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi'
$gmx_mpi grompp -f minim.mdp -c protein_solv_ions.gro -p topol.top -o em.tpr

In [ ]:
%%bash
gmx_mpi='/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi'
$gmx_mpi mdrun -v -nb gpu -deffnm em

In [ ]:
%%bash
gmx_mpi='/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi'
echo -e "10" | $gmx_mpi energy -f em.edr -o potential.xvg

In [ ]:
!xmgrace potential.xvg

# Step 5: equilibrando la temperatura

In [ ]:
%%bash
gmx_mpi='/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi'
$gmx_mpi grompp -f nvt.mdp -c em.gro -r em.gro -p topol.top -o nvt.tpr

In [ ]:
%%bash
gmx_mpi='/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi'
$gmx_mpi mdrun -v -nb gpu -deffnm nvt

In [ ]:
%%bash
gmx_mpi='/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi'
echo -e "16" | $gmx_mpi energy -f nvt.edr -o temperature.xvg

In [ ]:
!xmgrace temperature.xvg 

# Step 6: equilibrando la presion

In [ ]:
%%bash
gmx_mpi='/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi'
$gmx_mpi grompp -f npt.mdp -c nvt.gro -r nvt.gro -t nvt.cpt -p topol.top -o npt.tpr

In [ ]:
%%bash
gmx_mpi='/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi'
$gmx_mpi mdrun -v -nb gpu -deffnm npt

In [ ]:
%%bash
gmx_mpi='/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi'
echo -e "18" | $gmx_mpi energy -f npt.edr -o pressure.xvg

In [ ]:
!xmgrace pressure.xvg

# Step 7: produciendo la dinamica

In [ ]:
%%bash
gmx_mpi='/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi'
$gmx_mpi grompp -f md.mdp -c npt.gro -t npt.cpt -p topol.top -o md_0_1.tpr

In [ ]:
%%bash
gmx_mpi='/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi'
$gmx_mpi mdrun -v -nb gpu -deffnm md_0_1

## Step 8: analizando los resultados

In [ ]:
%%bash
gmx="/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi"
echo -e "1" | gmx trjconv -f md_0_1.xtc -s md_0_1.tpr -pbc mol -ur compact -center -o outputfile.xtc
# Centrando las trajectorias

In [ ]:
%%bash 
gmx="/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi"
cat << EOF | $gmx rms -s md_0_1.tpr -f md_0_1.xtc -o rmsd.xvg -tu ns
4
4
EOF
# RMSD plot

In [ ]:
!xmgrace rmsd.xvg

In [ ]:
%%bash 
gmx="/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi"
cat << EOF | $gmx rmsf -s md_0_1.tpr -f md_0_1.xtc -res -o rmsf.xvg
4
4
EOF
# RMSF plot

In [ ]:
!xmgrace rmsf.xvg

In [ ]:
%%bash 
gmx="/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi"
echo -e '1' | $gmx trjconv -s md_0_1.tpr -f outputfile.xtc -dt 100 -o protein_centered.pdb
# Trajectorias en pdb

In [ ]:
!vmd trj.pdb

## Step 9: extendiendo la dinamica molecular

In [ ]:
%%bash 
gmx="/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi"
$gmx convert-tpr -s md_0_1.tpr -extend 1000 -o md_0_2.tpr

In [ ]:
%%bash
gmx='/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi'
$gmx mdrun -v -nb gpu -deffnm md_0_2

In [ ]:
%%bash
gmx='/home/pitts/Documents/gromacs-2018.3/build/bin/./gmx_mpi'
$gmx trjcat -f md_0_1.xtc md_0_2.xtc -o md_0_final.xtc -cat